In [1]:
import pandas as pd
df = pd.read_csv('../bank-model-experimentation/bank-full.csv')

In [2]:
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/514196
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
# Get or create the 'bank' feature group
bank_fg = fs.get_or_create_feature_group(
    name="bank_fg",
    version=1,
    primary_key=["age", "job", "martial", "education", "default", "balance", "housing", "loan", "contact", "day", "month", "duration", "campaign", "pdays", "previous", "poutcome", "y"],
    description="Bank feature group",
)
# Insert data info the feature group
bank_fg.insert(df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/514196/fs/510019/fg/610661


Uploading Dataframe: 0.00% |          | Rows 0/45211 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: bank_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/514196/jobs/named/bank_fg_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fc980e8a590>, None)

In [12]:
label_encoding_cols = ["job", "martial", "education", "default", "housing", "loan", "contact", "month", "poutcome"]
label_encoder = fs.get_transformation_function(name="label_encoder")
transform_function_dict = {col: label_encoder for col in label_encoding_cols}

In [13]:
# Feature View Creation
selected_features = bank_fg.select_all()
# Get or create the 'iris' feature view
bank_fv = fs.get_or_create_feature_view(
    name="bank_fv",
    version=1,
    labels=['y'],
    description="Bank feature view",
    query=selected_features,
    transformation_functions=transform_function_dict,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/514196/fs/510019/fv/bank_fv/version/1


In [4]:
bank_fv = fs.get_feature_view("bank_fv", 1)

In [14]:
version, job = bank_fv.create_training_data(
    description='First Version',
    data_format='csv',
    # async creation in order not to wait till finish of the job
    write_options={"wait_for_job": True}
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/514196/jobs/named/bank_fv_1_create_fv_td_12032024174908/executions


In [15]:
features_df, labels_df = bank_fv.get_training_data(training_dataset_version=1)

In [18]:
features_df.head(5)

,age,job,martial,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,56,5,1,3,0,1214,0,1,1,7,1,105,1,-1,0,1
1,38,5,1,2,0,-367,1,1,1,12,2,354,1,-1,0,1
2,36,0,2,3,0,51,0,0,2,14,7,179,3,-1,0,1
3,33,5,1,2,0,1130,1,0,1,7,9,1063,1,-1,0,1
4,53,0,1,1,0,6408,0,0,0,6,0,29,1,-1,0,1
